# Accessing & working with the Digital Democracy Corpus: Comprehensive Proceedings of Four State Legislatures 2015-2018


This notebook will act as a guide for a person with minimal knowledge of coding to make use of the files and retrieve useful information from the corpus.


The dataset can be downloaded directly from this website: https://huggingface.co/datasets/iatpp/digitaldemocracy-2015-2018

This corpus is distributed under the cc-by-nc-sa-4.0 license. Please review before accessing. https://creativecommons.org/licenses/by-nc-sa/4.0/

Please cite as:

Khosmood, F., Dekhtyar, A., Ellwein, S., White, B., "The Digital Democracy Corpus: Comprehensive Proceedings of Four State Legislatures 2015-2018", Digital Humanities, Washington, DC, August 2024.

# Download files

The Digital Democracy Corpus can be downloaded from here: https://huggingface.co/datasets/iatpp/digitaldemocracy-2015-2018

The following code cell will downloaded the relevant files from the data repository and store them in the Colab runtime.

 Files in Colab runtime storage can be accessed by clicking on the folder button on the left sidebar. Double click on a file to view a preview of the file. Anything stored Colab runtime storage will disappear once the session is over. To save these files to your physical computer, right click on the file you'd like to save and click download.

In [1]:
!git clone https://huggingface.co/datasets/iatpp/digitaldemocracy-2015-2018

Cloning into 'digitaldemocracy-2015-2018'...
Updating files: 100% (3/3)
Updating files: 100% (3/3), done.




```
[# This is formatted as code](https://)
```

# Read the relevant files into Python Objects by set

In [2]:
import sys
print(sys.maxsize)
# 2147483647

9223372036854775807


In [3]:
import csv

max_size = 2147483647
csv.field_size_limit(max_size)

#This is where the unzipped corpus file is stored in Colab file storage
CORPUS_FILE_PATH = 'content/DH2024_Corpus_Release/'

#We have 4 state sets
VALID_STATES=["CA", "FL", "NY", "TX"]

#Each state set has 9 csv files for each session year
CSV_FILENAMES=['bills','committeeHearings', 'committeeRosters',
               'committees', 'hearings', 'legislature',
               'people','speeches','videos']

#Load data from CSVs into a Python object to reference later
#Input:
#  Required: file_name (type:String) (Ex: speeches, bills, etc)
#  Optional: states (type:List of Strings or None) (Ex: ["CA"], ["FL,TX"])
#     -If not specified (states=None), function returns data for all states
#  Optional: years (type:List of Ints or None) (Ex:[2018], [2015,2016])
#     -If not specified (years=None), function returns data for all valid years
#Output:
#  Payload (Type: Dict) (Ex: {column_headers:['pid','cid','date'], rows:[[0,2,2018],[2,1,2018]]})
def load_content(file_name, states=None, years=None):
  #Only accept valid states, Corpus only contains data on CA, FL, NY, and TX legislations
  if states is not None and not all(item in VALID_STATES for item in states):
    raise Exception("Invalid State Abbv(s), corpus only contains data on CA, FL, NY, and TX")
  #Only accept valid file names from corpus, like speeches, bills, etc.
  if file_name not in CSV_FILENAMES:
    raise Exception("Invalid filename, must be one of the 9 files provide")
  #Only accept years belonging to a valid legislative session. (2017-2018 for all states, 2015-2016 for CA)
  if years is not None and ((not all(item > 2015 for item in years) and "CA" not in states) or (not all(item <= 2018 for item in years))):
    raise Exception("""Data for requested year not included in corpus.
     Valid session_years are 2017 and 2018 for all states provided. 2015 and 2016 are valid years for CA.""")

  payload = {}
  header_row = True

  #If no states specified, retrieve relevant files for all valid states
  if states is None:
    states = VALID_STATES

  #If no years/session specified, retrieve data for all valid state legislative session years
  if years is None:
    if "CA" in states:
      years= [2015,2016,2017,2018]
    else:
      years = [2017,2018]

  #The following code block operates as follows:
  # For every state and year requested, read the relevant CSV file(s), then
  # load it into a python object (payload) which is returned to user
  for state in states:
    FILE_PATHS = []

    #Build the filepaths to the correct data location given the states and years provided
    #Years 2017 and 2018 are valid inputs that belong to the same 2017-2018 session
    if 2017 in years or 2018 in years:
      FILE_PATHS.append(CORPUS_FILE_PATH + state + "/2017-2018/CSV/" + file_name + ".csv")

    #CA has 2 valid legislative sessions (2015-2016 and 2017-2018)
    #This means the entirety of CA data is located in more than one folder, unlike other states.
    #Looping through a list of filepaths allows us to handle this corner case
    if state == "CA" and (2015 in years or 2016 in years):
      FILE_PATHS.append(CORPUS_FILE_PATH + state + "/2015-2016/CSV/" + file_name + ".csv")

    for FILE_PATH in FILE_PATHS:
      #Open the file to read
      with open(FILE_PATH, newline='') as csvfile:
        rows = csv.reader(csvfile, delimiter=',')
        #Read CSV row by row
        for row in rows:
          #The first row of every CSV we visit is the header row, containing the names for each column
          # We will add this to the payload only once, as every CSV we read after this will be the same headers
          if header_row:
            payload['column_headers'] = row
            #Sets up 'rows' in payload where we will store future records
            payload['rows'] = []
            header_row = False
            continue
          #Load CSV Into payload row by row
          payload['rows'].append(row)

  return payload

#Example Usage:
load_content("legislature", states= ["FL", "TX"], years=[2017])

{'column_headers': ['pid',
  'Lastname',
  'Firstname',
  'house',
  'district',
  'party',
  'biography',
  'twitter'],
 'rows': [['105013',
   'Ahern',
   'Larry',
   'House',
   '66',
   'Republican',
   'NaN',
   'NaN'],
  ['105014', 'Albritton', 'Ben', 'House', '56', 'Republican', 'NaN', 'NaN'],
  ['105015', 'Berman', 'Lori', 'House', '90', 'Democrat', 'NaN', 'NaN'],
  ['105015', 'Berman', 'Lori', 'Senate', '31', 'Democrat', 'NaN', 'NaN'],
  ['105016', 'Bileca', 'Michael', 'House', '115', 'Republican', 'NaN', 'NaN'],
  ['105017', 'Boyd', 'Jim', 'House', '71', 'Republican', 'NaN', 'NaN'],
  ['105018', 'Brodeur', 'Jason', 'House', '28', 'Republican', 'NaN', 'NaN'],
  ['105019', 'Corcoran', 'Richard', 'House', '37', 'Republican', 'NaN', 'NaN'],
  ['105020', 'Cruz', 'Janet', 'House', '62', 'Democrat', 'NaN', 'NaN'],
  ['105021', 'Diaz', 'Jose', 'House', '116', 'Republican', 'NaN', 'NaN'],
  ['105022', 'Drake', 'Brad', 'House', '5', 'Republican', 'NaN', 'NaN'],
  ['105023', 'Eisnaugle'

# Search for a Person

In [4]:
#Retrieve the first and last name belonging to a provided pid (person id)
#Input:
#  Required: pid (type:positive nonzero integer) (Ex: 102)
#  Optional: people (type:Dict or None)
#     -If not specified (people=None), function returns data for all states
#     -people is the output of load_content("people",...,...)
#     -If searching for a pid from a specific state or session, pass in
#         people=load_content("people", specific states, specific years)
#Output:
#   (Type: List[String, String]) (Ex: "John", "Smith")
#   If PID does not exist in database, returns None
def search_person_pid(pid,people=None):
  #These are the indexes/locations of information within each row
  PID_IDX=0
  LAST_NAME_IDX=1
  FIRST_NAME_IDX=2

  #If people data wasn't passed in, load data for all states and sessions
  if people is None:
    people=load_content("people")

  #Search through each row in people data until pid match is found
  for person in people['rows']:
    if person[PID_IDX] == str(pid):
      return person[FIRST_NAME_IDX],person[LAST_NAME_IDX]
  #No PID match found, return None
  return None

#Retrieve the first name, last name, and pid (person id) given a last and/or first name
#Input:
#  Required: last name (type:String) (Ex: Full last name "Smith" or Partial last name "Smi")
#  Optional: first name (type:String or None) (Ex: Full first name "Andrew" or Partial first name "drew")
#  Optional: case_sensitive (type:Boolean) (Default is True)
#  Optional: exact (Type:Boolean) (Default is True)
#     -Ex: If exact, searching for last name "James" will return matches for "James" and not names like "Jameson"
#     -Note: If exact=False and case sensitive=True, last name "Wein" will return matches for "Wein" and "Weinstein" but not "Ellwein"
#  Optional: people
#e      -If searching for a pid from a specific state or session, pass in
#         people=load_content("people", specific states, specific years)
#Output:
#   (Type: List[String, String, Int]) (Ex: "John", "Smith", 102)
#   If no matches exist does not exist in database, returns None
def search_person_keyword(last_name, first_name=None, case_sensitive=True, exact=True,people=None):
  #These are the indexes/locations of information within each row
  PID_IDX=0
  LAST_NAME_IDX=1
  FIRST_NAME_IDX=2
  #If people data wasn't passed in, load data for all states and sessions
  if people is None:
    people=load_content("people",years=[2017])

  hits = []

  #Search through every person in provided data
  for person in people['rows']:
    person_last_name = person[LAST_NAME_IDX]
    person_first_name = person[FIRST_NAME_IDX]
    person_pid = person[PID_IDX]

    #If not case sensitive, turn all strings uppercase for comparison
    if not case_sensitive:
      person_last_name = person_last_name.upper()
      person_first_name = person_first_name.upper()
      last_name = last_name.upper()
      if first_name:
        first_name = first_name.upper()

    #If looking for exact matches, check for strings that equal each other
    #If match is found, return the first, last name, and pid of matching record
    if exact and last_name == person_last_name:
      if first_name is None:
        hits.append(",".join((person[FIRST_NAME_IDX],person[LAST_NAME_IDX],person[PID_IDX])))
      elif first_name == person_first_name:
        hits.append(", ".join((person[FIRST_NAME_IDX],person[LAST_NAME_IDX],person[PID_IDX])))

    #If looking for partial matches, check for last/first names that contain provided keywords
    if not exact and last_name in person_last_name:
      if first_name is None:
        hits.append(",".join((person[FIRST_NAME_IDX],person[LAST_NAME_IDX],person[PID_IDX])))
      elif first_name in person_first_name:
        hits.append(", ".join((person[FIRST_NAME_IDX],person[LAST_NAME_IDX],person[PID_IDX])))

  #Return matches
  return hits

#Example usage
search_person_keyword('Hu', "an",exact=False, case_sensitive=True)

['Jillian, Huffman, 97476',
 'Jean, Hurst, 403',
 'Vanessa, Humphrey, 103780',
 'Morgan, Humphrey, 108686',
 'Frank, Huntington, 13962',
 'Jean, Hurst, 104687',
 'Dane, Hutchings, 2499',
 'Dean, Hutchins, 108896',
 'Dean, Hutchins, 108895',
 'Jean, Hurst, 109007',
 'Dean, Hutchings, 109505',
 'Delaney, Hunter, 263',
 'Alejandra, Huron, 111782',
 'Grant, Huey, 120803',
 'Jean, Hurst, 403',
 'Dane, Hutchings, 2499',
 'Dean, Hutchings, 109505',
 'Delaney, Hunter, 263',
 'Brian, Huffman, 153447',
 'Jonathan, Hughes, 157168',
 'Dean, Hutchins, 161955',
 'Sandra, Huerta, 162769',
 'Jean, Hurst, 109007',
 'Nathan, Huxley, 155824',
 'Shannon, Hunt, 156161',
 'Reagan, Hurt, 156778',
 'Brian, Hunt, 157871',
 'Susan, Husari, 3002',
 'Susan, Hunter, 146852',
 'Jane, Huang, 160048',
 'Randy, Hunt, 144708',
 'Hank, Hutchinson, 146065',
 'Blanche, Hurlbutt, 149211',
 'Joan, Huffman, 112236',
 'Bryan, Hughes, 112264',
 'Dan, Huberty, 112136',
 'Jonathan, Huss, 122042',
 'Jonathan, Huss, 121846',
 'Lua

# Find Bills & Hearings


In [5]:
#Retrieve the Hearing ID, legislative session, and hearing date where a provided Bill ID is discussed
#Input:
#  Required: BID (type:String) (Ex: Full 'FL_20170HB883' or partial '883' bill ID)
#  Optional: partial_match (Type:Boolean) (Default is False)
#  Optional: speeches
#e      -If searching for a bid from a specific state or session, pass in
#         speeches=load_content("speeches", specific states, specific years)
#Output:
#   (Type: List[String, Int, Int, Date]) (Ex: BID, HID, Session, Hearing Date)
#   If no matches exist does not exist in database, returns None
def find_bill_from_bid(bid, partial_match=False, speeches=None):
  #These are the indexes/locations of information within each row
  HID_IDX = 3
  BID_IDX = 4
  SESSION_IDX = 7
  DATE_IDX = 6
  #If data not provided, search through data from all states and sessions
  if speeches is None:
    speeches = load_content("speeches")
  #List of unique HIDS
  HIDS = []
  matches = []
  #For every speech, check if the BID being discussed matches the provided BID
  #If matching, add relevant BID, HID, Session, and Date to matches
  #Append HID to HIDS to prevent duplicates
  for row in speeches['rows']:
    if not partial_match and row[BID_IDX] == bid and row[HID_IDX] not in HIDS:
      matches.append([bid, row[HID_IDX], row[SESSION_IDX], row[DATE_IDX]])
      HIDS.append(row[HID_IDX])
    if partial_match and bid in row[BID_IDX] and row[HID_IDX] not in HIDS:
      matches.append([row[BID_IDX], row[HID_IDX], row[SESSION_IDX], row[DATE_IDX]])
      HIDS.append(row[HID_IDX])

  return matches
#Example usage
find_bill_from_bid('883', partial_match=True, speeches=load_content("speeches", states=["FL"]))

[['FL_20170HB883', '52345', '2017', '2017-03-21'],
 ['FL_20170HB883', '53163', '2017', '2017-03-28'],
 ['FL_20170HB883', '53120', '2017', '2017-04-06'],
 ['FL_20170HB883', '53057', '2017', '2017-05-01'],
 ['FL_20170HB883', '253074', '2017', '2018-01-16'],
 ['FL_20180HB883', '253578', '2017', '2018-02-22'],
 ['FL_20180HB883', '254077', '2017', '2018-03-02'],
 ['FL_20180HB883', '254139', '2017', '2018-03-05'],
 ['FL_20180HB883', '254438', '2017', '2018-03-07']]

In [6]:
#This is a helper function for cleaning up the XML text in bills.csv
from bs4 import BeautifulSoup
def clean_text(text):
    soup = BeautifulSoup(text, 'html.parser')
    for span in soup.find_all('span', class_='deletion'):
        span.decompose()

    text = soup.get_text(separator=' ', strip=True)
    return text

#helper function determines if keywords exist in text
def contains_keywords(text, keywords, all_keys):
    if all_keys:
      return all(keyword in text for keyword in keywords)
    else:
      return any(keyword in text for keyword in keywords)

In [7]:
#Retrieve the Hearing ID, Bill ID pairs that contains a keyword in Title or Body text
#Input:
#  Required: keyword (type:List[String]) (Ex: "education")
#  Required: target(type:String) ("Title" if searching through Title, else search through body text)
#  Optional: all_keys(type:Boolean) should matches contain all keys or just some
#  Optional: case_sensitive (Type:Boolean) (Default is True)
#  Optional: bills
#      -If searching through from a specific state or session, pass in
#         bills=load_content("bills", specific states, specific years)
#  Optional: speeches
#      -If searching through from a specific state or session, pass in
#         speeches=load_content("speeches", specific states, specific years)
#Output:
#   (Type: Dict{String: List[Int]}) (Ex: {BID: [HIDs,...]})
#   If no matches exist does not exist in database, returns {}
def bid_hid_from_keywords(keywords,all_keys=True,target='Title', case_sensitive=True, bills=None, speeches=None):
  #If searching for keyword in Title, set target location to title
  if target == 'Title':
    TARGET_IDX=9
  #Else, set target location to body text
  else:
    TARGET_IDX=11
  BID_IDX=0
  HID_IDX=1

  #BID matches stored here
  bids = []
  payload = {}

  #If bills or speeches not provided, search through data for all dates and sessions
  if bills is None:
    bills = load_content("bills")
  if speeches is None:
    speeches=load_content("speeches")

  if not case_sensitive:
    keywords = [keyword.upper() for keyword in keywords]

  #Go through every bill provided
  for row in bills['rows']:
    #If target text is XML, clean up text so we can search for valid matches
    if '<' in row[TARGET_IDX]:
      text = clean_text(row[TARGET_IDX])
    else:
      text = row[TARGET_IDX]
    if case_sensitive and contains_keywords(text,keywords,all_keys):
      bids.append(row[BID_IDX])
    if not case_sensitive and contains_keywords(text.upper(),keywords,all_keys):
      bids.append(row[BID_IDX])
  #For each BID found, find the HIDs where the bill is discussed
  for bid in bids:
    matches = find_bill_from_bid(bid, speeches=speeches)
    hids = []
    #Search through all BID HID matches to get list of HIDs
    for row in matches:
      hids.append(row[HID_IDX])
    #Store BID and associated list of HIDs in a dict
    payload[bid] = hids

  return payload

#To clear up confusion, simpley call this to search through bill Title
def bid_hid_from_title(keywords, all_keys=True, case_sensitive=True, bills=None, speeches=None):
  return bid_hid_from_keywords(keywords,all_keys=all_keys, target="Title", case_sensitive=case_sensitive, bills=bills, speeches=speeches)
#Or call this to search through bill Text
def bid_hid_from_bill_text(keywords, all_keys=True, case_sensitive=True, bills=None, speeches=None):
  return bid_hid_from_keywords(keywords,all_keys=all_keys, target="Text", case_sensitive=case_sensitive, bills=bills, speeches=speeches)

#Retrieve the Hearing ID, Bill ID pairs where a keyword is mentioned in a speech
#Input:
#  Required: keyword (type:List[String]) (Ex: "education")
#  Optional: all (type:Boolean) (default: True) All keywords in speech or some
#  Optional: case_sensitive (Type:Boolean) (Default is True)
#  Optional: speeches
#      -If searching through from a specific state or session, pass in
#         speeches=load_content("speeches", specific states, specific years)
#Output:
#   (Type: [String,Int]} (Ex: [BID, HID])
#   If no matches exist does not exist in database, returns []
def bid_hid_from_speech(keywords, all_keys=True,case_sensitive=True, speeches=None):
  BID_IDX=4
  HID_IDX=3
  TEXT_IDX=16

  #If speeches not provided, search through all speeches across states and sessions
  if speeches is None:
    speeches=load_content("speeches")

  if not case_sensitive:
    keywords = [keyword.upper() for keyword in keywords]

  #Look through every speech and if match found add BID, HID pairs to matches lost
  matches=[]
  for row in speeches['rows']:
    text = row[TEXT_IDX]
    pair = [row[BID_IDX], row[HID_IDX]]
    if case_sensitive and contains_keywords(text,keywords,all_keys) and pair not in matches:
      matches.append(pair)
    if not case_sensitive and contains_keywords(text.upper(),keywords,all_keys) and pair not in matches:
      matches.append(pair)
  return matches

#Retrieve the Hearing ID, Bill ID pairs where a specific person speaks
#Input:
#  Required: PID (type:Positive Int) (Ex: 1002)
#  Optional: speeches
#      -If searching through from a specific state or session, pass in
#         speeches=load_content("speeches", specific states, specific years)
#Output:
#   (Type: [String,Int]) (Ex: [BID, HID])
#   If no matches exist does not exist in database, returns []
def bid_hid_from_speaker(pid,speeches=None):
  BID_IDX=4
  HID_IDX=3
  PID_IDX=1

  #If speeches not provided, look through all speeches across all states and years
  if speeches is None:
    speeches=load_content("speeches")

  pid =str(pid)
  #Go through every speech, if PID match found, append [BID, HID] pair to matches list
  matches=[]
  for row in speeches['rows']:
    pair = [row[BID_IDX], row[HID_IDX]]
    if pid == row[PID_IDX] and pair not in matches:
      matches.append(pair)
  return matches

#Retrieve the Hearing ID, Bill ID pairs where a bill authored by specific person is discussed
#Input:
#  Required: PID (type:Positive Int) (Ex: 1002)
#  Optional: speeches
#      -If searching through from a specific state or session, pass in
#         speeches=load_content("speeches", specific states, specific years)
#  Optional: bills
#      -If searching through from a specific state or session, pass in
#         bills=load_content("bills", specific states, specific years)
#Output:
#   (Type: Dict{String: List[Int]}) (Ex: {BID: [HIDs,...]})
#   If no matches exist does not exist in database, returns []
def bid_hid_from_author_pid(author_pid, speeches=None, bills=None):
  AUTHOR_PID_IDX = 13
  BID_IDX = 0
  HID_IDX=1

  author_pid = str(author_pid)

  if bills is None:
    bills = load_content("bills")
  if speeches is None:
    speeches=load_content("speeches")

  bids = []
  payload = {}

  #Find all BIDs where provided PID is the author
  for row in bills['rows']:
    if author_pid == row[AUTHOR_PID_IDX]:
      bids.append(row[BID_IDX])
  #For each BID found, find the HIDs where the bill is discussed
  for bid in bids:
    matches = find_bill_from_bid(bid,partial_match=False, speeches=speeches)
    hids = []
    for row in matches:
      hids.append(row[HID_IDX])
    payload[bid] = hids

  return payload

#Retrieve the Hearing ID, Bill ID pairs for all bills/hearings from a specific date
#Input:
#  Required: date (type:String) (Ex: '2016-12-05')
#  Optional: speeches
#      -If searching through from a specific state or session, pass in
#         speeches=load_content("speeches", specific states, specific years)
#Output:
#   (Type: [String,Int]) (Ex: [BID, HID])
#   If no matches exist does not exist in database, returns []
def bid_hid_from_date(date, speeches=None):
  HEARING_DATE_IDX = 6
  BID_IDX=4
  HID_IDX=3

  #If speeches not specified, search through all states and sessions
  if speeches is None:
    speeches=load_content("speeches")

  matches = []
  bids = []
  for row in speeches['rows']:
    #If date matches the date in this row and the BID has not been encountered yet, update matches
    if date == row[HEARING_DATE_IDX] and row[BID_IDX] not in bids:
      matches.append([row[BID_IDX], row[HID_IDX]])
      bids.append(row[BID_IDX])

  return matches

In [8]:
#Example Usage
#Return hearings and bill pairs where 'hygeine' or 'health' appear in bill title in CA for all years
bid_hid_from_keywords(['hygiene','health'], all_keys=False, target='Title', case_sensitive=False, bills=load_content("bills",states=['CA']), speeches=load_content("speeches",states=["CA"]))

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 5531: character maps to <undefined>

In [17]:
#Example Usage
#Return the hearings and the discussed bills from a given day
bid_hid_from_date('2016-12-05', speeches=load_content("speeches",states=["CA"]))

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 1710: character maps to <undefined>

In [18]:
#Return the hearings and bills that PID 102 spoke about
#If the bill wasn't discussed, it will miss its pair
bid_hid_from_speaker(102, speeches=load_content("speeches", states=['CA']))

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 1710: character maps to <undefined>

In [19]:
#Return the hearings for which a bill that was authored by PID 102 was discussed in California
#If the bill wasn't discussed, it will miss its pair
bid_hid_from_author_pid(102, speeches=load_content("speeches", states=['CA']))

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 1710: character maps to <undefined>

In [20]:
#Example Usage
#Return the hearinngs and the discussed bills where someone strongly opposed a bill and was interrupted
bid_hid_from_speech(['bill', 'oppose', 'strongly','-'], all_keys = True,case_sensitive=True,speeches=load_content("speeches", states=["CA"]))

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 1710: character maps to <undefined>

In [21]:
from datetime import datetime, timedelta
#This is a helper function for calculating transcript times
def add_seconds(start_time, seconds_to_add):
    # Parse the start time
    time_obj = datetime.strptime(start_time, '%H:%M:%S')
    # Add the seconds
    new_time = time_obj + timedelta(seconds=seconds_to_add)
    # Format the new time back to string
    return new_time.strftime('%H:%M:%S')

In [36]:
#Retrieve Committee Name & ID, hearing date, list of videos of the hearing, and state for a given HID
#Input:
#  Required: HID (type:Positive Int) (Ex: 10003)
#  Optional: speeches
#      -If searching through from a specific state or session, pass in
#         speeches=load_content("speeches", specific states, specific years)
#Output:
#   Dictionary
#   If no matches exist does not exist in database, returns []
def get_metadata_hearing(hid, hearings=None, videos=None):
  HID_IDX=0
  CID_IDX=4
  CNAME_IDX=8
  HDATE_IDX=1
  STATE_IDX=3

  if hearings is None:
    hearings=load_content("hearings")
  hid = str(hid)

  hearingData=None
  for row in hearings['rows']:
    if hid == row[HID_IDX]:
      hearingData={'hid':row[HID_IDX],'cid':row[CID_IDX],'cname':row[CNAME_IDX],'hearing_date':row[HDATE_IDX],'state':row[STATE_IDX]}
  if hearingData is None:
    return {}

  #vids, videos = videos_from_hid(hid)
  #hearingData['vids'] = vids
  hearingData['videos'] = videos_from_hid(hid)

  return hearingData


#Retrieve the video ids and video URLs associated with a hearing
#Input:
#  Required: Hearing ID (type:Positive Int) (Ex: 10003)
#  Optional: videos
#      -If searching through from a specific state or session, pass in
#         videos=load_content("videos", specific states, specific years)
#Output:
#   (Type: List of[[Int,String]]) (Ex: [[0, "video0.mp4"], [1,"video1.mp4"]])
#   If no matches exist does not exist in database, returns []
def videos_from_hid(hid, videos=None):
  HID_IDX = 2
  VID_IDX = 0
  URL_IDX = 7

  if videos is None:
    videos=load_content("videos")

  vids = []
  video_files=[]
  hid=str(hid)

  for row in videos['rows']:
    if row[HID_IDX] == hid:
      vids.append((row[VID_IDX],row[URL_IDX])) #tuple of (video id, url)
  return vids

#Create transcript for a given bill discussion with metadata
#Input:
#  Required: HID (type:Positive Int) (Ex: 10003)
#  Required: BID (type: String)
#  Optional: speeches
#      -If searching through from a specific state or session, pass in
#         speeches=load_content("speeches", specific states, specific years)
#Output:
#   Dictionary and Transcript
#   If no matches exist does not exist in database, returns []
def get_hearing_transcript(hid, bid, speeches=None):
  PID_IDX=1
  BID_IDX=4
  HID_IDX=3
  VID_START_IDX = 9
  VID_END_IDX = 10
  LAST_NAME_IDX = 14
  FIRST_NAME_IDX = 15
  TEXT_IDX = 16
  STARTING_TIME_IDX = 11
  if speeches is None:
    speeches=load_content("speeches")

  hid=str(hid)

  lines = []
  for row in speeches['rows']:
    if hid == row[HID_IDX] and bid == row[BID_IDX]:
      offset_time = add_seconds("00:00:00", int(row[STARTING_TIME_IDX]))
      #line = "[{}] {} {} (pid {}) speaking: {}".format(offset_time,row[FIRST_NAME_IDX],row[LAST_NAME_IDX],row[PID_IDX],row[TEXT_IDX])
      line = {'video start':row[VID_START_IDX],'video end':row[VID_END_IDX],'offset':offset_time,'bid':row[BID_IDX],
              'first name':row[FIRST_NAME_IDX],'last name':row[LAST_NAME_IDX],'pid':row[PID_IDX],'text':row[TEXT_IDX]}
      lines.append(line)
  return lines

#Helper function to get bill discussion metadata
def bill_discussion_info(hid, bid, hearings=None, speeches=None, videos=None):
  return {"metadata":get_metadata_hearing(hid, hearings,videos), "transcript":get_hearing_transcript(hid, bid, speeches)}


# accepts the output from get_hearing_transcript() and prints a transceript.
def pprint_discussion(metadata, transcript_info):
  videos = {}
  for vid, url in metadata['videos']:
    videos[vid] = url

  print()
  print(f"] Discussion of {metadata['state']} {metadata['cname']} held on {metadata['hearing_date']}")
  print(f"] {len(videos.keys())} videos")
  print("] printing transcript: ")

  prev_video = -1
  for line in transcript_info:
    video = line['video start']
    if video != prev_video:
      print()
      print(f"] Discussing {line['bid']}")
      print(f"] Video: {videos[line['video start']] if line['video start'] in videos else line['video start']}")
      print()
      prev_video = video
    print(f"[{line['offset']}] {line['first name']} {line['last name']}: ")
    print(f"\t{line['text']}")
  print()


In [23]:
#Usage Example
#Get all videos from hearing 10003 list of (video id, video url) tuples
videos_from_hid(10003, videos=load_content("videos"))

[('16523',
  'https://videostorage-us-west.s3-us-west-2.amazonaws.com/videos/46674a437eda58890cbb935dda869e1a/46674a437eda58890cbb935dda869e1a.mp4#'),
 ('16524',
  'https://videostorage-us-west.s3-us-west-2.amazonaws.com/videos/f0781717dd6e46a92ec73c9a1accda64/f0781717dd6e46a92ec73c9a1accda64.mp4#'),
 ('16525',
  'https://videostorage-us-west.s3-us-west-2.amazonaws.com/videos/f57cef45cbcaebc5c73583e7673b7b5a/f57cef45cbcaebc5c73583e7673b7b5a.mp4#'),
 ('16526',
  'https://videostorage-us-west.s3-us-west-2.amazonaws.com/videos/bcdd60c43bfdbfc9fe50402c4764e228/bcdd60c43bfdbfc9fe50402c4764e228.mp4#')]

In [24]:
#Usage Example
#Get metadata from hearing 10003
get_metadata_hearing(10003, load_content("hearings", ["CA"]), load_content("speeches", ["CA"]))

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 1710: character maps to <undefined>

In [25]:
#Usage Example:
#Get metadata, videos, and recreate transcript from hearing 10003 and bill CA_201720180SR7
# discussion will contain all the information from bill CA_201720180SR7 in hearing id 10003
discussion = bill_discussion_info(10003, "CA_201720180SR7")
discussion

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 1710: character maps to <undefined>

In [26]:
# Using pprint_discussion we layout the transcript in a human readable format
pprint_discussion(discussion['metadata'], discussion['transcript'])

NameError: name 'discussion' is not defined

In [27]:
#Create transcript for speeches where a keyword is mentioned with metadata
#Input:
#  Required: Keywords (type:String) (Ex: "education")
#  Optional: All (type: Boolean) (Default is True)
#    -If All, returns only speeches that contain all keyword provided
#    -If not All, return any speeches that contain at least one keyword
#  Optional: PID (type: Int) -filter by speaker pid
#  Optional: Session (type:Int) -filter by session
#  Optional: HID (type: Int) -fiter by hearing
#  Optional: speeches
#      -If searching through from a specific state or session, pass in
#         speeches=load_content("speeches", specific states, specific years)
#Output:
#   Dictionary and Transcript
#   If no matches exist does not exist in database, returns []
def speeches_by_criteria(keywords,all_keys=True, pid=None,session=None,hid=None, date=None,speeches=None):
  BID_IDX=4
  HID_IDX=3
  PID_IDX=1
  TEXT_IDX=16
  SESSION_IDX = 7
  DATE_IDX=6

  if speeches is None:
    speeches=load_content("speeches")

  transcripts = []

  for row in speeches['rows']:
    if contains_keywords(row[TEXT_IDX],keywords,all_keys):

      #IF optional field was provided, check for match, if doesn't match, skip
      if pid and row[PID_IDX]!=str(pid):
        continue
      if session and row[SESSION_IDX] != str(session):
        continue
      if hid and row[HID_IDX] != str(hid):
        continue
      if date and row[DATE_IDX] != date:
        continue
      firstname, lastname = search_person_pid(row[PID_IDX])
      line = "{} {} (pid {}) speaking on {} during hearing {} session {}: {}".format(firstname, lastname, row[PID_IDX], row[DATE_IDX], row[HID_IDX], row[SESSION_IDX], row[TEXT_IDX])
      transcripts.append(line)

  return transcripts


In [28]:
#Find all times PID 102 talked about health or hygeine
speeches_by_criteria(["health", "hygeine"],all_keys=False, pid=99, speeches=load_content("speeches", states=["CA"]))

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 1710: character maps to <undefined>